# Importing the libraries

In [1]:
import numpy as np
import random
import os
import tensorflow as tf
from tensorflow import keras
import platform
from tensorflow.keras import layers, optimizers, losses, models, Input, Model
import time # Per misurare il tempo di training
from tensorflow.keras.callbacks import EarlyStopping # Per l'early stopping
import matplotlib.pyplot as plt
from tqdm import tqdm # Per mostrare una barra di progresso

# Print the HW Specs.

In [2]:
print("--- Dettagli dell'Architettura Hardware della Sessione Colab ---\n")

# --- 1. Dettagli CPU ---
print("--- Dettagli CPU ---")
!lscpu
print("\n")

# --- 2. Dettagli RAM (Memoria) ---
print("--- Dettagli RAM (Memoria) ---")
!cat /proc/meminfo | grep MemTotal
print("\n")

# --- 3. Dettagli Spazio su Disco ---
print("--- Dettagli Spazio su Disco ---")
!df -h /
print("\n")

# --- 4. Dettagli Acceleratore Hardware (GPU/TPU) ---
print("--- Dettagli Acceleratore Hardware (GPU/TPU) ---")
try:
    tpu_address = os.environ.get('COLAB_TPU_ADDR')
    if tpu_address:
        resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=tpu_address)
        tf.config.experimental_connect_to_cluster(resolver)
        tf.tpu.experimental.initialize_tpu_system(resolver)
        print(f"Tipo Acceleratore: TPU (indirizzo: {tpu_address})")
        print("Dispositivi TPU disponibili:")
        for device in tf.config.list_logical_devices('TPU'):
            print(f"  - {device.name}")
    else:
        gpus = tf.config.list_physical_devices('GPU')
        if gpus:
            print(f"Tipo Acceleratore: GPU")
            for gpu in gpus:
                print(f"  - Dispositivo GPU rilevato: {gpu.name}")
            print("\nDettagli GPU specifici (da `!nvidia-smi`):")
            !nvidia-smi
        else:
            print("Tipo Acceleratore: Nessuna GPU o TPU rilevata (in uso CPU)")

except Exception as e:
    print(f"Si è verificato un errore durante la rilevazione dell'acceleratore: {e}")
    print("Tentativo di rilevare i dispositivi TensorFlow standard:")
    devices = tf.config.list_logical_devices()
    if devices:
        for device in devices:
            print(f"  - Dispositivo rilevato: {device.name}, Tipo: {device.device_type}")
    else:
        print("Nessun dispositivo TensorFlow rilevato.")

print("\n--- Analisi Dettagli Hardware Completata ---")

--- Dettagli dell'Architettura Hardware della Sessione Colab ---

--- Dettagli CPU ---
Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   2
  On-line CPU(s) list:    0,1
Vendor ID:                GenuineIntel
  Model name:             Intel(R) Xeon(R) CPU @ 2.00GHz
    CPU family:           6
    Model:                85
    Thread(s) per core:   2
    Core(s) per socket:   1
    Socket(s):            1
    Stepping:             3
    BogoMIPS:             4000.46
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht sysc
                          all nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xt
                          opology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq
                           ssse3 fma cx16 pcid sse4_1 sse4

# Connect To Gdrive to store the datasets created.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Experimental Parameters Setting

In [7]:
# --- 1. Parametri di Simulazione di Base ---
BETA = 5  # Fattore di spreading (lunghezza della sequenza caotica per simbolo)
M = 4      # Numero di sottoportanti
X0_INITIAL = 0.9 # Stato iniziale per le mappe caotiche
SNR_TRAIN_RANGE_DB = [0, 20] # SNR range for training

# Dataset sizes (adjust as needed for computational resources)
NUM_SYMBOLS_TRAIN = 60000
NUM_SYMBOLS_VALIDATION = 20000

CHANNEL_TYPE = 'awgn'
L_FADING = 1 # Parametro per Rayleigh fading

# --- 1. Impostazione del Seed Globale all'inizio dello script ---
# Questo è il punto chiave per la riproducibilità di TUTTO ciò che segue.
MASTER_RANDOM_SEED = 42
np.random.seed(MASTER_RANDOM_SEED)
random.seed(MASTER_RANDOM_SEED) # Imposta anche il seed per la libreria 'random' di Python se la usi
tf.random.set_seed(MASTER_RANDOM_SEED)
os.environ['PYTHONHASHSEED'] = str(MASTER_RANDOM_SEED) # Per operazioni basate su hash (es. ordine dei dizionari)
os.environ['TF_DETERMINISTIC_OPS'] = '1' # Forza operazioni deterministiche in TensorFlow 2.x


# Defining signals-related functions.

In [5]:
# --- 2. Generazione delle Mappe Caotiche ---
def bernoulli_map(x0, length):
    """Genera una sequenza dalla mappa di Bernoulli shift."""
    sequence = np.zeros(length)
    x = x0
    for i in range(length):
        x = (2 * x) % 1
        sequence[i] = x
    return sequence

def logistic_map(x0, length, rho=3.6):
    """Genera una sequenza dalla mappa Logistica."""
    sequence = np.zeros(length)
    x = x0
    for i in range(length):
        x = rho * x * (1 - x)
        sequence[i] = x
    return sequence

# --- 3. Modulazione Multi-Carrier e Simulazione del Canale ---
def mc_modulate(chaotic_sequence, M):
    """
    Simula la modulazione multi-carrier replicando la sequenza su M "sottoportanti".
    """
    return np.tile(chaotic_sequence, (M, 1)) # M x BETA

def add_awgn(signal, snr_db):
    """Aggiunge rumore AWGN a un segnale complesso."""
    # Calcola la potenza del segnale (assumendo potenza media)
    signal_power = np.mean(np.abs(signal)**2)

    # Converte SNR da dB a lineare
    snr_linear = 10**(np.array(snr_db) / 10) # Explicitly treat snr_db as scalar

    # Calcola la potenza del rumore necessaria
    # snr_linear = signal_power / noise_power
    noise_power = signal_power / snr_linear

    # Genera rumore complesso (parte reale e immaginaria) con potenza corretta
    # np.sqrt(noise_power / 2) perché il rumore è distribuito su Re e Im
    noise_amplitude = np.sqrt(noise_power / 2)
    # print(f"Noise amplitude shape: {noise_amplitude.shape}") # Debug print
    noise = noise_amplitude * (np.random.randn(*signal.shape) + 1j * np.random.randn(*signal.shape))
    return signal + noise

def add_rayleigh_fading(signal, snr_db, L=1):
    """
    Aggiunge fading di Rayleigh (L percorsi) e rumore AWGN a un segnale.
    Il segnale viene normalizzato dopo il fading per mantenere la potenza media prima del rumore.
    Ora applica fading indipendente per ciascuna "sottoportante" (riga) se il segnale è 2D.
    """
    if signal.ndim == 1: # Caso single carrier, trasforma in 2D per coerenza
        num_subcarriers = 1
        signal_to_fade = signal[np.newaxis, :]
    else: # Caso multi-carrier (M x BETA)
        num_subcarriers = signal.shape[0]
        signal_to_fade = signal

    faded_signal_sum_all_subcarriers = np.zeros_like(signal_to_fade, dtype=complex)

    for _ in range(L):
        # Genera coefficienti di fading di Rayleigh indipendenti per ciascuna sottoportante
        # h sarà una colonna (num_subcarriers, 1) per il broadcast corretto
        h = (np.random.randn(num_subcarriers, 1) + 1j * np.random.randn(num_subcarriers, 1)) / np.sqrt(2)
        faded_signal_sum_all_subcarriers += h * signal_to_fade

    # Normalizza la potenza del segnale dopo il fading per mantenere la potenza del segnale originaria
    # prima di aggiungere AWGN e quindi l'SNR calcolato correttamente.
    original_signal_power = np.mean(np.abs(signal_to_fade)**2)
    current_faded_power = np.mean(np.abs(faded_signal_sum_all_subcarriers)**2)

    if current_faded_power > 0:
        faded_signal_normalized = faded_signal_sum_all_subcarriers * np.sqrt(original_signal_power / current_faded_power)
    else:
        faded_signal_normalized = np.zeros_like(signal_to_fade, dtype=complex)

    if signal.ndim == 1: # Se l'input originale era 1D, ritorna 1D
        faded_signal_normalized = faded_signal_normalized.flatten()

    return add_awgn(faded_signal_normalized, snr_db)

def form_receiver_input(received_signals_mc, beta):
    """
    Prende i segnali ricevuti dalle M sottoportanti (M x BETA) e forma l'input 2xbeta.
    Media dei segnali ricevuti su tutte le M sottoportanti.
    """
    averaged_signal = np.mean(received_signals_mc, axis=0) # Risultato: (BETA,) complex

    # Separa parte reale e immaginaria e concatena per formare un vettore 2*beta
    input_vector = np.concatenate([np.real(averaged_signal), np.imag(averaged_signal)])
    return input_vector

# Generate datasets function.

In [10]:
def generate_dataset(num_symbols_train, num_symbols_val,
                     snr_train_range_db,
                     beta, M, channel_type='AWGN', L=1):
    """
    Genera i dataset completi per training e validation,
    e un test set separato per ogni punto SNR specificato.
    La riproducibilità è garantita dall'impostazione globale del seed di NumPy
    all'inizio dello script.

    Args:
        num_symbols_train (int): Numero di simboli per il training set.
        num_symbols_val (int): Numero di simboli per il validation set.
        snr_range_db (list): Lista dei valori di SNR in dB.
        beta (float): Parametro per le mappe caotiche.
        M (int): Ordine della modulazione.
        channel_type (str): Tipo di canale ('AWGN' o 'Rayleigh').
        L (int): Parametro per il canale Rayleigh.

    Returns:
        tuple: (X_train, y_train, X_val, y_val, X_test_by_snr, y_test_by_snr)
               - X_train, y_train: Dataset di training.
               - X_val, y_val: Dataset di validation.
               - X_test_by_snr (dict): Dizionario di array NumPy per i dati di test,
                                       dove le chiavi sono i valori di SNR.
               - y_test_by_snr (dict): Dizionario di array NumPy per le etichette di test,
                                       dove le chiavi sono i valori di SNR.
    """

    print(f"--- Generazione Dataset per Canale: {channel_type} ---")

    def _generate_single_split(num_symbols, current_snr_config, split_name, fixed_snr=None):
        X_data = np.zeros((num_symbols, 2 * beta))
        y_labels = np.zeros(num_symbols, dtype=int)

        for i in tqdm(range(num_symbols), desc=f"Generating {channel_type} {split_name} Dataset"):
            # Nel caso reale la mappa da scegliere dipende dal simbolo che voglio trasmettere non da np.random.rand()
            # Se voglio trasmettere uno 0 allora scelgo la mappa di bernoulli. altrimenti logistic. (e la label deve essere scelta di conseguenza)
            map_type = 'bernoulli' if np.random.rand() < 0.5 else 'logistic'
            label = 0 if map_type == 'bernoulli' else 1

            if fixed_snr is not None:
                snr_db = fixed_snr
            elif isinstance(current_snr_config, list):
                snr_db = np.random.uniform(current_snr_config[0], current_snr_config[1])
            else: # Singolo valore SNR
                snr_db = current_snr_config

            x0_current = np.random.rand()

            chaotic_sequence = bernoulli_map(x0_current, beta) if map_type == 'bernoulli' else logistic_map(x0_current, beta)
            modulated_signal = mc_modulate(chaotic_sequence, M)

            if channel_type == 'AWGN':
                received_signal_mc = add_awgn(modulated_signal, snr_db)
            elif channel_type == 'Rayleigh':
                received_signal_mc = add_rayleigh_fading(modulated_signal, snr_db, L=L)
            else:
                raise ValueError("channel_type deve essere 'AWGN' o 'Rayleigh'")

            X_data[i] = form_receiver_input(received_signal_mc, beta)
            y_labels[i] = label
        return X_data, y_labels

    # --- Generazione Training Set ---
    print("\nInizio generazione Training Set...")
    # Il range SNR per il training è 11-15 dB, come indicato nel paper [cite: 115]
    # L'SNR per il training è una variabile casuale all'interno del range specificato [cite: 114]
    X_train, y_train = _generate_single_split(num_symbols_train, snr_train_range_db, "Train")
    print("Training Set generato.")

    # --- Generazione Validation Set ---
    print("\nInizio generazione Validation Set...")
    X_val, y_val = _generate_single_split(num_symbols_val, snr_train_range_db, "Validation")
    print("Validation Set generato.")


    return X_train, y_train, X_val, y_val


# Generate the dataset for AWGN channel.

In [12]:
# Path di base su Google Drive
base_path = '/content/drive/MyDrive/Academic/2025/Conferences/INFOCOM2026/GitHub/AWGN/dataset'

sequence_lengths = [5, 10, 15, 20, 30]
for M in sequence_lengths:
    print(f"\n📏 Lunghezza sequenza caotica M = {M}")

    # 1. Dataset con SNR da 0 a 20 dB
    SNR_TRAIN_RANGE_DB = [0, 20]
    X_train_awgn, y_train_awgn, X_val_awgn, y_val_awgn = generate_dataset(
        num_symbols_train=NUM_SYMBOLS_TRAIN,
        num_symbols_val=NUM_SYMBOLS_VALIDATION,
        snr_train_range_db=SNR_TRAIN_RANGE_DB,
        beta=BETA,
        M=M,
        channel_type='AWGN',
        L=L_FADING
    )

    np.savez(os.path.join(base_path, f'training_0-20_SNR_{M}_BETA.npz'),
             X_train=X_train_awgn,
             y_train=y_train_awgn)
    np.savez(os.path.join(base_path, f'validation_0-20_SNR_{M}_BETA.npz'),
             X_val=X_val_awgn,
             y_val=y_val_awgn)

    # 2. Dataset con SNR tra 11 e 15 dB
    SNR_TRAIN_RANGE_DB = [11, 15]
    X_train_awgn, y_train_awgn, X_val_awgn, y_val_awgn = generate_dataset(
        num_symbols_train=NUM_SYMBOLS_TRAIN,
        num_symbols_val=NUM_SYMBOLS_VALIDATION,
        snr_train_range_db=SNR_TRAIN_RANGE_DB,
        beta=BETA,
        M=M,
        channel_type='AWGN',
        L=L_FADING
    )

    np.savez(os.path.join(base_path, f'training_11-15_SNR_{M}_BETA.npz'),
             X_train=X_train_awgn,
             y_train=y_train_awgn)
    np.savez(os.path.join(base_path, f'validation_11-15_SNR_{M}_BETA.npz'),
             X_val=X_val_awgn,
             y_val=y_val_awgn)

    print(f"✅ Dataset M={M} salvati.")

print("\n✅ Tutti i dataset sono stati generati e salvati correttamente.")


📏 Lunghezza sequenza caotica M = 5
--- Generazione Dataset per Canale: AWGN ---

Inizio generazione Training Set...


Generating AWGN Train Dataset: 100%|██████████| 60000/60000 [00:04<00:00, 14499.79it/s]


Training Set generato.

Inizio generazione Validation Set...


Generating AWGN Validation Dataset: 100%|██████████| 20000/20000 [00:01<00:00, 10911.64it/s]


Validation Set generato.
--- Generazione Dataset per Canale: AWGN ---

Inizio generazione Training Set...


...

Validation Set generato.
--- Generazione Dataset per Canale: AWGN ---

Inizio generazione Training Set...


Generating AWGN Train Dataset: 100%|██████████| 60000/60000 [00:05<00:00, 10439.90it/s]


Training Set generato.

Inizio generazione Validation Set...


Generating AWGN Validation Dataset: 100%|██████████| 20000/20000 [00:01<00:00, 11996.21it/s]


Validation Set generato.
✅ Dataset M=30 salvati.

✅ Tutti i dataset sono stati generati e salvati correttamente.


# Generate the dataset for Rayleigh channel.

In [13]:
# Path di base su Google Drive
base_path = '/content/drive/MyDrive/Academic/2025/Conferences/INFOCOM2026/GitHub/Rayleigh/dataset'

sequence_lengths = [5, 10, 15, 20, 30]
for M in sequence_lengths:
    print(f"\n📏 Lunghezza sequenza caotica M = {M}")

    # 1. Dataset con SNR da 0 a 20 dB
    SNR_TRAIN_RANGE_DB = [0, 20]
    X_train, y_train, X_val, y_val = generate_dataset(
        num_symbols_train=NUM_SYMBOLS_TRAIN,
        num_symbols_val=NUM_SYMBOLS_VALIDATION,
        snr_train_range_db=SNR_TRAIN_RANGE_DB,
        beta=BETA,
        M=M,
        channel_type='Rayleigh',
        L=L_FADING
    )

    np.savez(os.path.join(base_path, f'training_0-20_SNR_{M}_BETA.npz'),
             X_train=X_train,
             y_train=y_train)
    np.savez(os.path.join(base_path, f'validation_0-20_SNR_{M}_BETA.npz'),
             X_val=X_val,
             y_val=y_val)

    # 2. Dataset con SNR tra 11 e 15 dB
    SNR_TRAIN_RANGE_DB = [11, 15]
    X_train, y_train, X_val, y_val = generate_dataset(
        num_symbols_train=NUM_SYMBOLS_TRAIN,
        num_symbols_val=NUM_SYMBOLS_VALIDATION,
        snr_train_range_db=SNR_TRAIN_RANGE_DB,
        beta=BETA,
        M=M,
        channel_type='Rayleigh',
        L=L_FADING
    )

    np.savez(os.path.join(base_path, f'training_11-15_SNR_{M}_BETA.npz'),
             X_train=X_train,
             y_train=y_train)
    np.savez(os.path.join(base_path, f'validation_11-15_SNR_{M}_BETA.npz'),
             X_val=X_val,
             y_val=y_val)

    print(f"✅ Dataset M={M} salvati.")

print("\n✅ Tutti i dataset sono stati generati e salvati correttamente.")


📏 Lunghezza sequenza caotica M = 5
--- Generazione Dataset per Canale: Rayleigh ---

Inizio generazione Training Set...


Generating Rayleigh Train Dataset: 100%|██████████| 60000/60000 [00:07<00:00, 7863.42it/s]


Training Set generato.

Inizio generazione Validation Set...


Generating Rayleigh Validation Dataset: 100%|██████████| 20000/20000 [00:02<00:00, 6783.38it/s]


Validation Set generato.
--- Generazione Dataset per Canale: Rayleigh ---

Inizio generazione Training Set...


...

Validation Set generato.
--- Generazione Dataset per Canale: Rayleigh ---

Inizio generazione Training Set...


Generating Rayleigh Train Dataset: 100%|██████████| 60000/60000 [00:09<00:00, 6245.93it/s]


Training Set generato.

Inizio generazione Validation Set...


Generating Rayleigh Validation Dataset: 100%|██████████| 20000/20000 [00:02<00:00, 7386.39it/s]

Validation Set generato.
✅ Dataset M=30 salvati.

✅ Tutti i dataset sono stati generati e salvati correttamente.
